In [1]:
# Top level configuration options
ALLOW_ALL = False
ALLOWED_CATEGORIES = ["bicycling", "running", "sports", "transportation", "walking"]

In [11]:
from json import load
import numpy as np
from pathlib import Path

p = (Path().resolve() / "../pose_data/annotations.json").resolve()

# Load the annotation data
annotations = load(p.open())

# Parse out the categories we care about

valid_imgs = []
for idx, act in enumerate(annotations["act"]):
    if ALLOW_ALL or act["cat_name"] in ALLOWED_CATEGORIES:
        valid_imgs.append(idx)

# Split the data into train and test
TEST_PERCENT = 0.1
n_test = int(np.floor(len(valid_imgs) * TEST_PERCENT))

np.random.shuffle(valid_imgs)  # TODO: Fix random seed for reproducibility
test, train = valid_imgs[:n_test], valid_imgs[n_test:]

print(
    f"Total dataset size: {len(valid_imgs)}, training with: {len(train)}, testing with: {len(test)}"
)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jdobry\\repos\\pose_data\\annotations.json'